In [1]:
# cc (type), 17353 (act), 865403 (section)
case_type = "cc"
act = 17353
section = 865403

In [2]:
import dask.dataframe as dd
import pandas as pd

In [3]:
from dask.distributed import Client

client = Client()
client

/home/advin4603/PrecogRecruitmentTask/env/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42935 instead
  warnings.warn(
2023-01-20 13:08:23,704 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-g7srprxg', purging
2023-01-20 13:08:23,705 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-fl6mt3ze', purging
2023-01-20 13:08:23,705 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-hxmf5i87', purging
2023-01-20 13:08:23,705 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-blqwn9yw', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:42935/status,
Dashboard: http://127.0.0.1:42935/status,Workers: 4
Total threads: 8,Total memory: 15.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32825,Workers: 4
Dashboard: http://127.0.0.1:42935/status,Total threads: 8
Started: Just now,Total memory: 15.52 GiB
Comm: tcp://127.0.0.1:42655,Total threads: 2
Dashboard: http://127.0.0.1:37761/status,Memory: 3.88 GiB
Nanny: tcp://127.0.0.1:35855,


In [4]:
fc = dd.read_parquet("Parquet/features_classes/")
fc = fc.compute()
fc = fc.sample(frac=1)
fc

,ddl_case_id,duration,year,state_code,dist_code,court_no,female_defendant,female_petitioner,female_adv_def,female_adv_pet,disp_name_s,fd,fp,fad,fap,dp
1140,17-18-03-205900014452015,495,2015,17,18,3,0 male,-9998 unclear,0,-9998,judgement,2,0,2,0,8
2069,17-11-02-205900169332010,1319,2010,17,11,2,0 male,-9998 unclear,-9999,-9999,judgement,2,0,1,1,8
1568,29-04-07-202101003982011,142,2011,29,4,7,1 female,1 female,0,0,referred to lok adalat,3,2,2,2,10
1207,02-21-04-202102006132013,359,2013,2,21,4,0 male,-9998 unclear,-9999,0,referred to lok adalat,2,0,1,2,10
996,04-02-05-202905014642012,106,2012,4,2,5,0 male,-9998 unclear,-9999,-9999,fine,2,0,1,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,04-05-10-202900010622014,142,2014,4,5,10,0 male,-9998 unclear,-9999,-9999,convicted,2,0,1,1,5
490,17-22-05-205900006002018,359,2018,17,22,5,0 male,-9998 unclear,-9999,-9998,judgement,2,0,1,0,8
490,29-07-10-202100002162018,596,2017,29,7,10,0 male,1 female,-9999,-9999,confession,2,2,1,1,4
1538,29-02-03-202108006522013,86,2013,29,2,3,0 male,-9998 unclear,-9999,-9999,transferred,2,0,1,1,11


In [5]:
from torch.utils.data import Dataset
import torch

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")


class DispositionDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.duration = dataframe["duration"]
        self.year = dataframe["year"].iloc
        self.state_code = dataframe["state_code"].iloc
        self.dist_code = dataframe["dist_code"].iloc
        self.court_no = dataframe["court_no"].iloc
        self.female_def = dataframe["fd"].iloc
        self.female_pet = dataframe["fp"].iloc
        self.female_adv_def = dataframe["fad"].iloc
        self.female_adv_pet = dataframe["fap"].iloc
        self.disposition = dataframe["dp"].iloc

    def __len__(self):
        return len(self.duration)

    def __getitem__(self, idx):
        duration = self.duration.iloc[idx]
        year = self.year[idx]
        state_code = self.state_code[idx]
        dist_code = self.dist_code[idx]
        court_no = self.court_no[idx]
        female_def = self.female_def[idx]
        female_pet = self.female_pet[idx]
        female_adv_def = self.female_adv_def[idx]
        female_adv_pet = self.female_adv_pet[idx]
        disposition = self.disposition[idx]

        return torch.Tensor((duration / 5000, year - 2010, state_code, dist_code, court_no, female_def, female_pet,
                             female_adv_def, female_adv_pet)), int(disposition)


train_data = DispositionDataset(fc[:-20000])
test_data = DispositionDataset(fc[-20000:])


Using cpu device


In [21]:
# learning_rate = .001
# weight_decay = 1e-2
batch_size = 100
epochs = 10

In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [23]:
from torch import nn
import optuna


class DispositionPredictor(nn.Module):
    def __init__(self):
        super(DispositionPredictor, self).__init__()
        n_layers = 1
        layers = []
        in_features = 9
        for i in range(n_layers):
            out_features = 18
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            in_features = out_features
        layers.append(nn.Linear(in_features, 34))
        self.linear_relu_stack = nn.Sequential(*layers)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return self.softmax(logits)



In [24]:

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)
        pred = model(torch.Tensor(X))
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [35]:
def train_and_evaluate(param, model, trial, train_dataloader, test_dataloader):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = getattr(torch.optim, param['optimizer'])(model.parameters(), lr=param['learning_rate'], weight_decay=param['weight_decay'])
    size = len(train_dataloader.dataset)
    test_size = len(test_dataloader.dataset)
    for epoch_num in range(epochs):
        for batch_num, (X, y) in enumerate(train_dataloader):
            # Compute prediction and loss
            X = X.to(device)
            y = y.to(device)
            pred = model(torch.Tensor(X))
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if batch_num % 100 == 0:
                loss, current = loss.item(), batch_num * len(X)
                print(f"[Training] loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


        num_batches = len(test_dataloader)
        test_loss, correct = 0, 0

        with torch.no_grad():
            for X, y in test_dataloader:
                X = X.to(device)
                y = y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= num_batches
        correct /= test_size
        print(f"{epoch_num}:[Test] Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        trial.report(correct, epoch_num)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return correct


def objective(trial):
    params = {
        'learning_rate': 0.014,
        # 'weight_decay': 0.0005310881230135281,
        'weight_decay': 0.0003,
        # 'weight_decay': 0,
        'optimizer': "Adam"
    }
    model = DispositionPredictor().to(device)
    accuracy = train_and_evaluate(params, model, trial, train_dataloader, test_dataloader)

    return accuracy

study = optuna.create_study(direction="maximize",
                            sampler=optuna.samplers.TPESampler(),
                            pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=1)

[I 2023-01-20 13:20:55,420] A new study created in memory with name: no-name-7ffd5057-85cc-495d-ba87-bab3a83c136c


[Training] loss: 3.517288  [    0/146653]
[Training] loss: 3.158055  [10000/146653]
[Training] loss: 3.201867  [20000/146653]
[Training] loss: 3.213402  [30000/146653]
[Training] loss: 3.062239  [40000/146653]
[Training] loss: 3.153908  [50000/146653]
[Training] loss: 3.214943  [60000/146653]
[Training] loss: 3.138347  [70000/146653]
[Training] loss: 3.067544  [80000/146653]
[Training] loss: 3.046072  [90000/146653]
[Training] loss: 3.022060  [100000/146653]
[Training] loss: 3.183343  [110000/146653]
[Training] loss: 3.117736  [120000/146653]
[Training] loss: 3.090921  [130000/146653]
[Training] loss: 3.156540  [140000/146653]
0:[Test] Error: 
 Accuracy: 48.1%, Avg loss: 3.096753 

[Training] loss: 3.073262  [    0/146653]
[Training] loss: 3.095058  [10000/146653]
[Training] loss: 3.156502  [20000/146653]
[Training] loss: 3.173889  [30000/146653]
[Training] loss: 3.006078  [40000/146653]
[Training] loss: 3.079429  [50000/146653]
[Training] loss: 3.135770  [60000/146653]
[Training] loss

[I 2023-01-20 13:23:01,528] Trial 0 finished with value: 0.48065 and parameters: {}. Best is trial 0 with value: 0.48065.


9:[Test] Error: 
 Accuracy: 48.1%, Avg loss: 3.095501 



In [ ]:

best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))